In [5]:

# conda install -c conda-forge xesmf
# pip install cftime

import dask
# from dask_gateway import Gateway

import s3fs
import gcsfs
import xarray as xr
import pandas as pd
import numpy as np
import xesmf as xe
import matplotlib.pyplot as plt
import zarr
import cftime
import tqdm
import datetime

xr.set_options(display_style='html')
%matplotlib inline
%config InlineBackend.figure_format = 'retina' 
plt.rcParams['figure.figsize'] = 12, 6

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [6]:
### data loading function
fs = s3fs.S3FileSystem(anon=True, default_fill_cache=False)
gcs = gcsfs.GCSFileSystem(token='anon')

In [7]:
s3_url = 's3://cmip6-pds/cmip6-zarr-consolidated-stores.csv'
gg_url = 'https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv'

In [8]:
### read data index
df = pd.read_csv(gg_url)
df.tail(5)

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
493419,CMIP,AS-RCEC,TaiESM1,historical,r2i1p1f1,Amon,va,gn,gs://cmip6/CMIP6/CMIP/AS-RCEC/TaiESM1/historic...,NaN,20210416
493420,CMIP,AS-RCEC,TaiESM1,historical,r2i1p1f1,Amon,zg,gn,gs://cmip6/CMIP6/CMIP/AS-RCEC/TaiESM1/historic...,NaN,20210416
493421,CMIP,AS-RCEC,TaiESM1,historical,r2i1p1f1,Amon,ua,gn,gs://cmip6/CMIP6/CMIP/AS-RCEC/TaiESM1/historic...,NaN,20210416
493422,CMIP,AS-RCEC,TaiESM1,historical,r2i1p1f1,Amon,ta,gn,gs://cmip6/CMIP6/CMIP/AS-RCEC/TaiESM1/historic...,NaN,20210416
493423,CMIP,AS-RCEC,TaiESM1,historical,r2i1p1f1,Amon,wap,gn,gs://cmip6/CMIP6/CMIP/AS-RCEC/TaiESM1/historic...,NaN,20210416


In [9]:
### find the model that has both historical and scenario records 
df_his_585 = df.query("variable_id == 'pr' & experiment_id == 'historical' & table_id == 'day' & member_id == 'r1i1p1f1' & activity_id=='CMIP'")
df_ssp_585 = df.query("variable_id == 'pr' & experiment_id == 'ssp585'     & table_id == 'day' & member_id == 'r1i1p1f1'")
df_merged_ssp_585  = pd.merge(df_his_585, df_ssp_585, on =['source_id','member_id','table_id','variable_id','grid_label'], how='inner')

df_his_245 = df.query("variable_id == 'pr' & experiment_id == 'historical' & table_id == 'day' & member_id == 'r1i1p1f1' & activity_id=='CMIP'")
df_ssp_245 = df.query("variable_id == 'pr' & experiment_id == 'ssp245'     & table_id == 'day' & member_id == 'r1i1p1f1'")
df_merged_ssp_245  = pd.merge(df_his_245, df_ssp_245, on =['source_id','member_id','table_id','variable_id','grid_label'], how='inner')

df_his_126 = df.query("variable_id == 'pr' & experiment_id == 'historical' & table_id == 'day' & member_id == 'r1i1p1f1' & activity_id=='CMIP'")
df_ssp_126 = df.query("variable_id == 'pr' & experiment_id == 'ssp126'     & table_id == 'day' & member_id == 'r1i1p1f1'")
df_merged_ssp_126  = pd.merge(df_his_126, df_ssp_126, on =['source_id','member_id','table_id','variable_id','grid_label'], how='inner')

### get intersection
historical_url = list(set(df_merged_ssp_245['zstore_x']).intersection(set(df_merged_ssp_126['zstore_x'])).intersection(set(df_merged_ssp_585['zstore_x'])))

### get ssp 585
ssp126_url = df_merged_ssp_126[df_merged_ssp_126['zstore_x'].isin(historical_url)]['zstore_y'].tolist()

In [10]:
print('number of models: ', len(ssp126_url))
ssp126_url.sort()
ssp126_url[:5]

number of models:  24


['gs://cmip6/CMIP6/ScenarioMIP/BCC/BCC-CSM2-MR/ssp126/r1i1p1f1/day/pr/gn/v20190315/',
 'gs://cmip6/CMIP6/ScenarioMIP/CCCR-IITM/IITM-ESM/ssp126/r1i1p1f1/day/pr/gn/v20201112/',
 'gs://cmip6/CMIP6/ScenarioMIP/CCCma/CanESM5/ssp126/r1i1p1f1/day/pr/gn/v20190429/',
 'gs://cmip6/CMIP6/ScenarioMIP/CMCC/CMCC-CM2-SR5/ssp126/r1i1p1f1/day/pr/gn/v20200717/',
 'gs://cmip6/CMIP6/ScenarioMIP/CMCC/CMCC-ESM2/ssp126/r1i1p1f1/day/pr/gn/v20210126/']

### read and store SSP126 data

In [22]:
# dictonary to collect data
ds_126_dictonary = {}
for i in tqdm.tqdm(range(len(ssp126_url))):
    zstore = ssp126_url[i]

   ### load data, use_cftime=True helps line up with the date format
    try:
        mapper = fs.get_mapper(zstore.replace("gs://cmip6", "s3://cmip6-pds"))
        ds_126 = xr.open_zarr(mapper, consolidated=True, decode_times=True, use_cftime=True)
    except KeyError:
        mapper = gcs.get_mapper(zstore.replace("s3://cmip6-pds", "gs://cmip6"))
        ds_126 = xr.open_zarr(mapper, consolidated=True, decode_times=True, use_cftime=True)

    
    min_date = ds_126.time.values.min()
    max_date = ds_126.time.values.max()
    if min_date.year > 2100:
        print('model ' + str(i) + ' min year is ' + str(min_date.year))
        continue
    if max_date.year < 2015:
        print('model ' + str(i) + ' max year is ' + str(max_date.year))
        continue
        
    ### keep data within 2015-2100 
    if (min_date.year == 2015) & ((min_date.month ==  1)) & ((min_date.day ==  1))\
     & (max_date.year == 2100) & ((max_date.month == 12)) & ((max_date.day == 31)):
        min_year = ds_126.time.values.min().year
        max_year = ds_126.time.values.max().year
        list_of_year = range(min_year, max_year + 1)
    else:
        ds_126 = ds_126.sel(time = slice('2015', '2100'))
        min_year = ds_126.time.values.min().year
        max_year = ds_126.time.values.max().year
        list_of_year = range(min_year, max_year + 1)
        if min(list_of_year) != 2015:
            print(str(i) + ': min year of the model is not 2015! It is: ', str(min(list_of_year)))
        if max(list_of_year) != 2100:
            print(str(i) + ': max year of the model is not 2100! It is: ', str(max(list_of_year)))

    ### drop model with 360 days
    for y in list_of_year:
        days = len(ds_126.sel(time = str(y)).time.values)
        if days == 360:
            print('model ' + str(i) + ' has 360 days in a year!!')
            break
            
    else:
        ### Drop leap day
        ds_126 = ds_126.where((ds_126['time.month'] != 2) | (ds_126['time.day'] != 29), drop=True)
        days = len(ds_126.sel(time = str(y)).time.values)
        if days != 365:
            print('model ' + str(i) + ' year ' + str(y) + ' does not have 365 days! It has ' + str(days) + ' days!!' )
        ds_126_dictonary[i] = ds_126

  8%|▊         | 2/24 [00:03<00:40,  1.86s/it]

1: max year of the model is not 2100! It is:  2099


 92%|█████████▏| 22/24 [00:35<00:02,  1.36s/it]

model 21 has 360 days in a year!!


100%|██████████| 24/24 [00:37<00:00,  1.57s/it]


In [24]:
### Model 20 has duplicated date with same value
print('before:')
print(ds_126_dictonary[21].sel(time = '2031-10-21', lon = 0, lat = 90).pr.values)

_, index = np.unique(ds_126_dictonary[20]['time'], return_index=True)
ds_126_dictonary[21] = ds_126_dictonary[20].isel(time=index)
print('')
print('after:')
print(ds_126_dictonary[21].sel(time = '2031-10-21', lon = 0, lat = 90).pr.values)

before:


KeyError: 21

### Check if any model is missing lat/lon bounds

In [25]:
for i in tqdm.tqdm(ds_126_dictonary.keys()):
    try:
        ds_126_dictonary[i].lat_bnds
    except AttributeError:
        print('ssp126 model ' + str(i) + ' does not have lat bounds!!')
    try:
        ds_126_dictonary[i].lon_bnds
    except AttributeError:
        print('ssp126 model ' + str(i) + ' does not have lon bounds!!')

100%|██████████| 23/23 [00:00<00:00, 1925.95it/s]

ssp126 model 13 does not have lat bounds!!
ssp126 model 13 does not have lon bounds!!


### Regrid + cftime to numpy.dateframe

In [26]:
ds_out = xr.Dataset({'lat': (['lat'], np.arange(-89.5, 90.0, 1.0)),
                     'lat_bnds' : (['lat', 'bnds'], np.array([[x, x + 1] for x in range(-90,90)])),
                     'lon': (['lon'], np.arange(0.5, 360.0, 1.0)),
                     'lon_bnds' : (['lon', 'bnds'], np.array([[x, x + 1] for x in range(0,360)])),
                    })

for i in tqdm.tqdm(ds_126_dictonary.keys()):
    regridder = xe.Regridder(ds_126_dictonary[i], ds_out, 'nearest_s2d', reuse_weights=False)
    ds_126_dictonary[i] = regridder(ds_126_dictonary[i]) 
    regridder._grid_in = None
    regridder._grid_out = None
    ds_126_dictonary[i]['time'] = ds_126_dictonary[i].indexes['time'].to_datetimeindex().normalize()

print('SSP126 - Done!')

  0%|          | 0/23 [00:00<?, ?it/s]

Overwrite existing file: nearest_s2d_160x320_180x360.nc 
 You can set reuse_weights=True to save computing time.
using dimensions ('lat', 'lon') from data variable pr as the horizontal dimensions for this dataset.


  4%|▍         | 1/23 [00:00<00:11,  1.96it/s]

Overwrite existing file: nearest_s2d_94x192_180x360.nc 
 You can set reuse_weights=True to save computing time.
using dimensions ('lat', 'lon') from data variable pr as the horizontal dimensions for this dataset.


  9%|▊         | 2/23 [00:00<00:10,  2.07it/s]

...

Overwrite existing file: nearest_s2d_180x288_180x360.nc 
 You can set reuse_weights=True to save computing time.
using dimensions ('lat', 'lon') from data variable pr as the horizontal dimensions for this dataset.


 96%|█████████▌| 22/23 [00:11<00:00,  1.99it/s]

Overwrite existing file: nearest_s2d_96x192_180x360.nc 
 You can set reuse_weights=True to save computing time.
using dimensions ('lat', 'lon') from data variable pr as the horizontal dimensions for this dataset.


100%|██████████| 23/23 [00:11<00:00,  1.98it/s]

SSP126 - Done!


### Mean & Std Calculation for SSP126

In [27]:
ds_126_dictonary[1]

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 180, lon: 360, time: 31025)
Coordinates:
  * time       (time) datetime64[ns] 2015-01-01 2015-01-02 ... 2099-12-31
    time_bnds  (time, bnds) object dask.array<chunksize=(15512, 2), meta=np.ndarray>
  * lon        (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
Dimensions without coordinates: bnds
Data variables:
    pr         (time, lat, lon) float64 dask.array<chunksize=(874, 180, 360), meta=np.ndarray>
Attributes:
    regrid_method:  nearest_s2d

In [30]:
# model 3 only has data till 2099, so we fill in 2100 data with nan
ds_126_3 = xr.concat([ds_126_dictonary[1], ds_126_dictonary[0].sel(time = slice('2099-01-01','2100-12-31'))], dim = 'time')
ds_126_3['pr'] = xr.where(ds_126_3.time >= np.datetime64("2099-01-01T12:00:00"), np.nan, ds_126_3.pr)

ds_126_dictonary[1] = ds_126_dictonary[0]

In [31]:
ds_126_dictonary

{0: <xarray.Dataset>
 Dimensions:    (bnds: 2, lat: 180, lon: 360, time: 31390)
 Coordinates:
   * time       (time) datetime64[ns] 2015-01-01 2015-01-02 ... 2100-12-31
     time_bnds  (time, bnds) object dask.array<chunksize=(15695, 2), meta=np.ndarray>
   * lon        (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
   * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
 Dimensions without coordinates: bnds
 Data variables:
     pr         (time, lat, lon) float64 dask.array<chunksize=(244, 180, 360), meta=np.ndarray>
 Attributes:
     regrid_method:  nearest_s2d, 1: <xarray.Dataset>
 Dimensions:    (bnds: 2, lat: 180, lon: 360, time: 31390)
 Coordinates:
   * time       (time) datetime64[ns] 2015-01-01 2015-01-02 ... 2100-12-31
     time_bnds  (time, bnds) object dask.array<chunksize=(15695, 2), meta=np.ndarray>
   * lon        (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
   * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 8

In [32]:
# create new dimension "model"
ds_126_temp = ds_126_dictonary[0].expand_dims({'model':list(ds_126_dictonary.keys())})
for i in tqdm.tqdm(ds_126_dictonary.keys()):
    ds_126_temp['pr'] = xr.where(ds_126_temp.model == i, ds_126_dictonary[i].pr, ds_126_temp.pr)

100%|██████████| 23/23 [00:00<00:00, 54.46it/s]


In [33]:
ds_126_output = ds_126_temp.mean('model').rename_vars({'pr':'pr_mean'})
ds_126_output['std'] = ds_126_temp.std('model').pr
ds_126_output

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 180, lon: 360, time: 31390)
Coordinates:
  * time       (time) datetime64[ns] 2015-01-01 2015-01-02 ... 2100-12-31
    time_bnds  (time, bnds) object dask.array<chunksize=(15695, 2), meta=np.ndarray>
  * lon        (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
Dimensions without coordinates: bnds
Data variables:
    pr_mean    (time, lat, lon) float64 dask.array<chunksize=(128, 180, 360), meta=np.ndarray>
    std        (time, lat, lon) float64 dask.array<chunksize=(128, 180, 360), meta=np.ndarray>

In [ ]:
ds_126_output.to_netcdf('ssp126_daily_pr.nc')

Unclosed connection
client_connection: Connection<ConnectionKey(host='cmip6-pds.s3.amazonaws.com', port=443, is_ssl=True, ssl=None, proxy=None, proxy_auth=None, proxy_headers_hash=None)>
Unclosed connection
client_connection: Connection<ConnectionKey(host='cmip6-pds.s3.amazonaws.com', port=443, is_ssl=True, ssl=None, proxy=None, proxy_auth=None, proxy_headers_hash=None)>
Unclosed connection
client_connection: Connection<ConnectionKey(host='cmip6-pds.s3.amazonaws.com', port=443, is_ssl=True, ssl=None, proxy=None, proxy_auth=None, proxy_headers_hash=None)>
Unclosed connection
client_connection: Connection<ConnectionKey(host='cmip6-pds.s3.amazonaws.com', port=443, is_ssl=True, ssl=None, proxy=None, proxy_auth=None, proxy_headers_hash=None)>
Unclosed connection
client_connection: Connection<ConnectionKey(host='cmip6-pds.s3.amazonaws.com', port=443, is_ssl=True, ssl=None, proxy=None, proxy_auth=None, proxy_headers_hash=None)>
Unclosed connection
client_connection: Connection<ConnectionKey(h

In [2]:
!aws s3 cp ssp126_daily.nc s3://aer-astd-mcclim/ssp126/ssp126_daily.nc

upload: ./ssp126_daily.nc to s3://aer-astd-mcclim/ssp126/ssp126_daily.nc


### SSP126 Monthly

In [16]:
ds_126_output['CDD'] = xr.where(((ds_126_output.tas_mean - 273.15) * 9/5 + 32) > 65, ((ds_126_output.tas_mean - 273.15) * 9/5 + 32) - 65, 0)
ds_126_output['HDD'] = xr.where(((ds_126_output.tas_mean - 273.15) * 9/5 + 32) < 65, 65 - ((ds_126_output.tas_mean - 273.15) * 9/5 + 32), 0)
ds_126_output_monthly = ds_126_output.resample(time = 'M').sum(dim = 'time')[['CDD', 'HDD']]
ds_126_output_monthly

<xarray.Dataset>
Dimensions:  (lat: 180, lon: 360, time: 1032)
Coordinates:
  * time     (time) datetime64[ns] 2015-01-31 2015-02-28 ... 2100-12-31
    height   float64 2.0
  * lon      (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 355.5 356.5 357.5 358.5 359.5
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
Data variables:
    CDD      (time, lat, lon) float64 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    HDD      (time, lat, lon) float64 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>

### Output netcdf file

In [ ]:
ds_126_output_monthly.to_netcdf('ssp126_monthly.nc')

In [3]:
!aws s3 cp ssp126_monthly.nc s3://aer-astd-mcclim/ssp126/ssp126_monthly.nc

upload: ./ssp126_monthly.nc to s3://aer-astd-mcclim/ssp126/ssp126_monthly.nc


### Verify the result

In [24]:
t = 0
n = 0
for i in range(0, 25):
    if i not in [4, 22, 3]:
        t += ds_126_dictonary[i].sel(time = '2100-11-15', lon = 1.5, lat = -72.5).tas.values
        n += 1
t/n

254.43862984397194

In [12]:
ds_daily = xr.open_dataset("ssp126_daily.nc")
ds_monthly = xr.open_dataset("ssp126_monthly.nc")

In [25]:
ds_daily.sel(time = '2100-11-15', lon = 1.5, lat = -72.5).tas_mean.values

array(254.43862984)

In [58]:
ds = ds_daily.sel(time = '2017-04', lon = 251.5, lat = -88.5)

In [59]:
ds['HDD'] = xr.where(((ds.tas_mean - 273.15) * 9/5 + 32) < 65, 65 - ((ds.tas_mean - 273.15) * 9/5 + 32), 0)
ds['CDD'] = xr.where(((ds.tas_mean - 273.15) * 9/5 + 32) > 65, ((ds.tas_mean - 273.15) * 9/5 + 32) - 65, 0)

In [60]:
print('CDD: ', ds.CDD.values.sum())
print('HDD: ', ds.HDD.values.sum())

CDD:  0.0
HDD:  3569.571373582921


In [61]:
d = ds_monthly.sel(time = '2017-04', lon = 251.5, lat = -88.5)
print('CDD: ', d.CDD.values)
print('HDD: ', d.HDD.values)

CDD:  [0.]
HDD:  [3569.57137358]
